# Статистика брокерів

# Завантаження та підготовка даних

У цьому блоці завантажено дані з Excel файлу, перетворено колонки `RPM`, `Broker` та `Status` у потрібний формат, а також перевіряно унікальні статуси та кількість брокерів у наборі даних.

In [1]:
import pandas as pd

df = pd.read_excel("../Loads 2024.xlsx", sheet_name="DATA")
df["RPM"] = pd.to_numeric(df["RPM"], errors="coerce")
df["Broker"] = df["Broker"].astype(str).str.strip()
df["Status"] = df["Status"].astype(str).str.strip().str.upper()

print("Унікальні статуси:", df["Status"].unique())
print(f"Загальна кількість брокерів у базі: {df['Broker'].nunique()}")

Унікальні статуси: ['BILLED' 'AVAILABLE' 'CANCELED' 'TONU']
Загальна кількість брокерів у базі: 1029


# Групування даних по брокерах та статусах

Підсумовування кількості вантажів для кожного брокера за кожним статусом (`BILLED`, `CANCELED`, `AVAILABLE` та іншими). Пропуски були заповнені нулями для уникнення помилок у подальшому аналізі. Додано стовпець із загальною кількістю вантажів по кожному брокеру.

In [2]:
status_by_broker = df.groupby(["Broker", "Status"]).size().unstack(fill_value=0).reset_index()

status_by_broker["Total"] = status_by_broker.drop(columns=["Broker"]).sum(axis=1)

if "CANCELED" in status_by_broker.columns:
    status_by_broker["Cancel Ratio"] = status_by_broker["CANCELED"] / status_by_broker["Total"]
else:
    status_by_broker["Cancel Ratio"] = 0

avg_rpm = df.groupby("Broker")["RPM"].mean().round(2)
status_by_broker["Avg RPM"] = status_by_broker["Broker"].map(avg_rpm)

status_by_broker

Status,Broker,AVAILABLE,BILLED,CANCELED,TONU,Total,Cancel Ratio,Avg RPM
0,0,0,1,0,0,1,0.0,2.82
1,12622501,0,1,0,0,1,0.0,1.46
2,29 LOG,0,2,0,0,2,0.0,2.71
3,3,0,1,0,0,1,0.0,1.75
4,3 RIVERS,1,4,0,0,5,0.0,2.30
...,...,...,...,...,...,...,...,...
1024,ZAAR,0,1,0,0,1,0.0,1.60
1025,ZENGISTICS,0,1,0,0,1,0.0,2.05
1026,ZIPLINE,1,2,0,0,3,0.0,2.52
1027,ZIPLINE LOG,0,3,0,0,3,0.0,2.31
